In [ ]:
from torch.quantization import quantize_fx
import torch
from mlp import ComplexMLP
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import torch.nn as nn

backend = 'fbgemm'
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
qconfig = torch.quantization.get_default_qconfig(backend)  
torch.backends.quantized.engine = backend
qconfig_dict = {"": torch.quantization.get_default_qconfig(backend)}

model_to_quantize = torch.load('whole_model.pth').to(device)
model_to_quantize.eval()


# Calibrate - Use representative (validation) data.
#验证数据
dh = pd.read_csv('dataset1.csv')
validation_data_columns = dh.columns[:27]
validation_label_columns = dh.columns[27:]  # 只选择第一个output的列
validation_data = torch.tensor(dh[validation_data_columns].values, dtype=torch.float32).to(device)
validation_labels = torch.tensor(dh[validation_label_columns].values, dtype=torch.float32).to(device)
for i in range(1000 - 1):
    validation_labels[i][0] = (validation_labels[i][0] * 99)
    validation_labels[i][1] = (validation_labels[i][1] * 99)
    validation_labels[i][2] = (validation_labels[i][2] * 99)
    validation_labels[i][3] = (validation_labels[i][3] * 99)
    validation_labels[i][4] = (validation_labels[i][4] * 99)
    validation_labels[i][5] = (validation_labels[i][5] * 99)
# 将数据转换为 PyTorch 的 DataLoader
val_dataset = TensorDataset(validation_data, validation_labels)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=True)


#测试量化前性能
with torch.no_grad():
    criterion = nn.MSELoss()
    validation_outputs = model_to_quantize(validation_data)
    validation_loss = criterion(validation_outputs, validation_labels)
print(f'Before quant Test Loss: {validation_loss.item():.4f}')


# Prepare
example_inputs = (torch.randn(1,1,1,27),)
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_dict, example_inputs)

with torch.inference_mode():
    # for _ in range(10):
    #     x = torch.rand(1, 2, 28, 28)
    #     model_prepared(x)
    for data,lable in tqdm(val_loader):
        model_prepared(data)

# quantize
model_quantized = quantize_fx.convert_fx(model_prepared)
print("quant completed. Saving model...")
torch.save(model_quantized, 'whole_model_quantized.pth')


#测试量化后性能
with torch.no_grad():
    criterion = nn.MSELoss()
    validation_outputs = model_quantized(validation_data)
    validation_loss = criterion(validation_outputs, validation_labels)
print(f'After quant Test Loss: {validation_loss.item():.4f}')